<h1> <b>Homework 2</b></h1>
<i>Alejandro J. Rojas<br>
ale@ischool.berkeley.edu<br>
W261: Machine Learning at Scale<br>
Week: 02<br>
Jan 26, 2016</i></li>

<h2>HW2.0.  </h2>
What is a race condition in the context of parallel computation? Give an example.
What is MapReduce?
How does it differ from Hadoop?
Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.

<h2>HW2.1. Sort in Hadoop MapReduce</h2>
Given as input: Records of the form '<'integer, “NA”'>', where integer is any integer, and “NA” is just the empty string.
Output: sorted key value pairs of the form '<'integer, “NA”'>' in decreasing order; what happens if you have multiple reducers? Do you need additional steps? Explain.

Write code to generate N  random records of the form '<'integer, “NA”'>'. Let N = 10,000.
Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers



# Data

In [8]:


import random

N = 10000                           ### for a sample size of N
random.seed(0)                      ### pick a random seed to replicate results


input_file = open("numcount.txt", "w") # writing file

for i in range(N):
    a = random.randint(0, 100)       ### Select a random integer from 0 to 100
    b = ''
    input_file.write(str(a))         
    input_file.write(b)
    input_file.write('\n')

   

input_file.close()


# Mapper

In [9]:
%%writefile mapper.py
#!/usr/bin/python
import sys

for line in sys.stdin:                         ### input comes from STDIN (standard input)
    
    number = line.strip()                      ### remove leading and trailing whitespace
    print ('%s\t%s' % (number, 1))             ### mapper out looks like 'number' \t 1

Overwriting mapper.py


In [10]:
!chmod +x mapper.py

# Reducer

In [11]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_number = None
current_count = 0
number = None
numlist = []

# input comes from STDIN
for line in sys.stdin:
    
    line = line.strip()                        ### remove leading and trailing whitespace
    line = line.split('\t')                    ### parse the input we got from mapper.py 
    number = line[0]                           ### integer generated randomly we got from mapper.py


    
    try:
        count = line[1]
        count = int(count)                      ### convert count (currently a string) to int
    except ValueError:                          ### if count was not a number then silently                         
        continue                                ### ignore/discard this line

                                                
                                                
    if current_number == number:                ### this IF-switch only works because Hadoop sorts map output
        current_count += count                  ### by key (here: number) before it is passed to the reducer
    else:
        if current_number:
            
            numlist.append((current_number,current_count))  ### store tuple in a list once totalize count per number

        current_count = count                   ### set current count
        current_number = number                 ### set current number


if current_number == number:                    ### do not forget to output the last word if needed!
    numlist.append((current_number,current_count))


toplist = sorted(numlist,key=lambda record: record[1], reverse=True) ### sort list from largest count to smallest
bottomlist = sorted(numlist,key=lambda record: record[1])  ### sort list from smalles to largest

print '%25s' %'TOP 10', '%25s' % '', '%28s' %'BOTTOM 10'
print '%20s' %'Number', '%10s' %'Count', '%20s' % '', '%20s' %'Number','%10s' %'Count'
for i in range (10):
    print '%20s%10s' % (toplist[i][0], toplist[i][1]),'%20s' % '', '%20s%10s' % (bottomlist[i][0], bottomlist[i][1])


Overwriting reducer.py


In [12]:
!chmod +x reducer.py

In [237]:
!echo "10 \n 10\n  5\n 6\n 8\n 9 \n 10 \n 9 \n 12 \n 21 \n 22 \n 23 \n 24 \n 25" | python mapper.py | sort -k1,1 | python reducer.py

  File "reducer.py", line 49
    print '%25s' %'TOP 10', '%25s' % '', '%28s' %'BOTTOM 10'
               ^
SyntaxError: Missing parentheses in call to 'print'
  File "mapper.py", line 27
    print '%s\t%s' % (word, 1)
                 ^
SyntaxError: invalid syntax


# Run numcount in Hadoop

<h2>start yarn  and hdfs</h2>

In [13]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh       ### start up yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh        ### start up dfs

starting yarn daemons
resourcemanager running as process 12434. Stop it first.
localhost: nodemanager running as process 13130. Stop it first.
16/01/25 20:32:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: namenode running as process 13875. Stop it first.
localhost: datanode running as process 14563. Stop it first.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: secondarynamenode running as process 15278. Stop it first.
16/01/25 20:33:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2> remove files from prior runs </h2>

In [14]:
!hdfs dfs -rm -r /user/venamax                           ### remove prior files

16/01/25 20:33:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 20:33:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/venamax


<h2> create folder</h2>

In [15]:
!hdfs dfs -mkdir -p /user/venamax                         ### create hdfs folder

16/01/25 20:33:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2> upload numcount.txt to hdfs</h2>

In [16]:
!hdfs dfs -put numcount.txt /user/venamax                 #### save source data file to hdfs            

16/01/25 20:33:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2> Hadoop streaming command </h2>

hadoop jar hadoopstreamingjarfile \

    -D stream.num.map.output.key.fields=n \
    -mapper mapperfile \
    -reducer reducerfile \
    -input inputfile \
    -output outputfile

In [17]:
!hadoop jar hadoop-*streaming*.jar -mapper mapper.py -reducer reducer.py -input numcount.txt -output numcountOutput

16/01/25 20:33:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 20:33:39 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/25 20:33:39 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/25 20:33:39 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/25 20:33:39 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/25 20:33:39 INFO mapreduce.JobSubmitter: number of splits:1
16/01/25 20:33:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local2145711988_0001
16/01/25 20:33:39 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/25 20:33:39 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/25 20:33:39 INFO mapreduce.Job: Running job: job_local2145711988_0001
16/01/25 20:33:3

<h2>show the results</h2>

In [18]:
!hdfs dfs -cat numcountOutput/part-00000

16/01/25 20:33:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                   TOP 10                                              BOTTOM 10	
              Number      Count                                    Number      Count	
                  77       125                                         9        76	
                  42       122                                        88        77	
                  90       117                                        70        83	
                  26       116                                        15        85	
                  80       115                                        35        85	
                  14       114                                        41        85	
                  86       113                                        59        85	
                  45       112                                        73        85	
            

<h2>stop yarn and hdfs </h2>

In [169]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/01/23 21:13:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/23 21:13:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2>=====================
END OF HW 2.1</h2>

<h2>HW2.2.  WORDCOUNT</h2>
Using the Enron data from HW1 and Hadoop MapReduce streaming, write the mapper/reducer job that  will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). Examine the word “assistance” and report its word count results.

 
CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l    
       8    
       #NOTE  "assistance" occurs on 8 lines but how many times does the token occur? 10 times! This is the number we are looking for!

# Mapper

In [297]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Alejandro J. Rojas
## Description: mapper code for HW2.2

import sys
import re



########## Collect user input  ###############
filename = 'enronemail_1h.txt'               
findwords = ['assistance', 'commercial', 'intelligence','call']
counts = {}
##filename = sys.argv[1]
##findwords = re.split(" ",sys.argv[2].lower())

for keyword in findwords:  ### Initialize to zero all keywords to find
    counts[keyword] = 0


## open the input file
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        record = re.split(r'\t+', line) 
        if len(record) == 4:                             ### Take only complete records
            for keyword in findwords:                    ### For each word to find
                for i in range (2,len(record)):          ### Compare it to the words from each email
                    bagofwords = re.split(" ",record[i]) ### Break each email records into words
                    for word in bagofwords:
                        neword = word.strip(',')         ### eliminate comas  
                        if keyword in neword:
                            counts[keyword] += 1


for keyword in findwords:                                 ### output results in the form:                   
    print ('%s\t%s'% (keyword, str(counts[keyword])))     ### word to find, count
            



Overwriting mapper.py


In [290]:
!chmod +x mapper.py

# Reducer

In [291]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None
words = {}

# input comes from STDIN


# input comes from STDIN
for line in sys.stdin:
    
    line = line.strip()                        ### remove leading and trailing whitespace
    line = line.split('\t')                    ### parse the input we got from mappe
    word = line[0]
    
    try:
        count = line[1]
        count = int(count)                      ### convert count (currently a string) to int
    except ValueError:                          ### if count was not a number then silently                         
        continue                                ### ignore/discard this line

                                                
                                                
    if current_word == word:                     ### this IF-switch only works because Hadoop sorts map output
        current_count += count                  ### by key (here: number) before it is passed to the reducer
    else:
        if current_word:
            
            words[current_word] = current_count  ### store tuple in a list once totalize count per number
  
        current_count = count                    ### set current count
        current_word = word                      ### set current word


if current_word == word:                         ### do not forget to output the last word if needed!
    words[current_word] = current_count 

for word in words:
    print ('We found %s' %word, ' on %s'%words[word] , 'occassions.')

Overwriting reducer.py


In [292]:
!chmod +x reducer.py

# Run numcount in Hadoop

<h2>start yarn  and hdfs</h2>

In [114]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh       ### start up yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh        ### start up dfs

starting yarn daemons
resourcemanager running as process 12434. Stop it first.
localhost: nodemanager running as process 13130. Stop it first.
16/01/25 16:46:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: namenode running as process 13875. Stop it first.
localhost: datanode running as process 14563. Stop it first.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: secondarynamenode running as process 15278. Stop it first.
16/01/25 16:46:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2> remove files from prior runs</h2>

In [293]:
!hdfs dfs -rm -r /user/venamax                           ### remove prior files

16/01/25 19:23:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:23:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/venamax


<h2> create folder</h2>

In [294]:
!hdfs dfs -mkdir -p /user/venamax                         ### create hdfs folder

16/01/25 19:23:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2> upload enronemail_1h.txt to hdfs</h2>

In [295]:
!hdfs dfs -put enronemail_1h.txt /user/venamax            #### save source data file to hdfs         

16/01/25 19:23:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2> Hadoop streaming command </h2>

In [296]:
!hadoop jar hadoop-*streaming*.jar -mapper mapper.py -reducer reducer.py -input enronemail_1h.txt -output wordcountOutput

16/01/25 19:23:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:23:48 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/25 19:23:48 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/25 19:23:48 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/25 19:23:48 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/25 19:23:49 INFO mapreduce.JobSubmitter: number of splits:1
16/01/25 19:23:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local722687238_0001
16/01/25 19:23:49 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/25 19:23:49 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/25 19:23:49 INFO mapreduce.Job: Running job: job_local722687238_0001
16/01/25 19:23:49 

<h2>show the results</h2>

In [ ]:
!hdfs dfs -cat wordcountOutput/part-00000

<h2>stop yarn and hdfs </h2>

In [16]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/01/25 15:11:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/25 15:11:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2>=====================
END OF HW 2.2</h2>

# Mapper

In [235]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Alejandro J. Rojas
## Description: mapper code for HW2.2

import sys
import re



########## Collect user input  ###############
filename = sys.argv[1]
findwords = re.split(" ",sys.argv[2].lower())



with open (filename, "r") as myfile:

    for line in myfile.readlines():
        line = line.strip()
        record = re.split(r'\t+', line)                    ### Each email is a record with 4 components
                                                           ### 1) ID 2) Spam Truth 3) Subject 4) Content
        if len(record)==4:                                 ### Take only complete records
            for i in range (2,len(record)):                ### Starting from Subject to the Content               
                bagofwords = re.split(" " | "," ,record[i])### Collect all words present on each email                
                for word in bagofwords: 
                    flag=0
                    if word in findwords:
                        flag=1                    
                    print '%s\t%s\t%s\t%s\t%s' % (word, 1,record[0], record[1],flag) 
                                                            ### output: word, 1, id, spam truth and flag
 

Overwriting mapper.py


In [236]:
!chmod +x mapper.py

# Reducer

In [ ]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys
from itertools import groupby



current_word, word = None, None                 
current_wordcount, current_spam_wordcount, current_ham_wordcount = 0,0,0
current_id, record_id = None, None          
current_y_true, y_true = None, None
current_flag, flag = None,None

sum_records, sum_spamrecords, sum_hamrecords = 0,0,0
sum_spamwords, sum_hamwords = 0,0
flagged_words = []

emails={} #Associative array to hold email data
words={} #Associative array for word data


# input comes from STDIN
for line in sys.stdin:
    
    line = line.strip()                        ### remove leading and trailing whitespace
    line = line.split('\t')                    ### parse the input we got from mapper.py 
    word = line[0]                             ### word we get from mapper.py


    
    try:
        count = line[1]
        count = int(count)                      ### convert count (currently a string) to int
        email = line[2]                         ### id that identifies each email
        y_true = line[3]
        y_true = int(y_true)                    ### spam truth as an integer
        flag = line[4]
        flag = int(flag)                        ### flags if word is in the user specified list
    except ValueError:                          ### if count was not a number then silently                         
        continue                                ### ignore/discard this line

                                                
                                                
    if current_word == word:                    ### this IF-switch only works because Hadoop sorts map output
        current_count += count                  ### by key (here: word) before it is passed to the reducer
        
        if current_word not in words.keys():
            words[current_word]={'ham_count':0,'spam_count':0,'flag':flag}
        if email not in emails.keys():
            emails[current_email]={'y_true':y_true,'word_count':0,'words':[]}
            sum_records +=1
            if y_true == 1:
                sum_spamrecords +=1
            else
                sum_hamrecords +=1
    
        if y_true == 1:                         ### if record where word is located is a spam
            current_spamcount += count         ### add to spam count of that word
            sum_spamwords += 1
        else:
            current_hamcount += count          ### if not add to ham count of thet word
            sum_hamwords +=1 

        emails[current_email]['word_count'] += 1
        emails[current_email]['words'].append(current_word)### store words in email      
        


    else:
        if current_word:
            if flag==1 and current_word not in flagged_words:
                flagged_words.append(current_word)                
            
            words[current_word]['flag'] = flag  ### denote if current word is a word specified by the user list
            words[current_word]['spam_count'] += current_spamcount ### update spam count for current word 
            words[current_word]['ham_count'] += current_hamcount ### update ham count for current word

  
            

        current_count = count                   ### set current count
        current_spamcount, current_hamcount = 0,0 ### initialize spam and ham wordcount
        current_word = word                     ### set current number
        current_email = email                   ### set current id of email
        current_y_true = y_true                 ### set current spam truth
        current_flag = flag                     ### set current flag 
        


if current_word == word:                       ### do not forget to output the last word if needed!
    emails[current_email]['word_count'] += 1
    emails[current_email]['words'].append(current_word)### store words in email  
    words[current_word]['flag'] = flag  ### denote if current word is a word specified by the user list
    words[current_word]['spam_count'] += current_spamcount ### update spam count for current word 
    words[current_word]['ham_count'] += current_hamcount ### update ham count for current word    
    


#Calculate stats for entire corpus
prior_spam= sum_spamrecords/sum_records     
prior_ham=sum_hamrecords/sum_records
vocab_count=len(words)#number of unique words in the total vocabulary
            
for k,word in words.iteritems():
    #These versions calculate conditional probabilities WITH Laplace smoothing.  
    #word['p_spam']=(word['spam_count']+1)/(spam_word_count+vocab_count)
    #word['p_ham']=(word['ham_count']+1)/(ham_word_count+vocab_count)
    
    #Compute conditional probabilities WITHOUT Laplace smoothing
    word['p_spam']=(word['spam_count'])/(sum_spamwords)
    word['p_ham']=(word['ham_count'])/(sum_hamwords)

#At this point the model is now trained, and we can use it to make our predictions

print '%30s' %'ID', '%10s' %'TRUTH', '%10s' %'CLASS', '%20s' %'CUMULATIVE ACCURACY'
miss, sample_size = 0,0 

for j,email in emails.iteritems():
    
    #Log versions - no longer used
    #p_spam=log(prior_spam)
    #p_ham=log(prior_ham)
    
    p_spam=prior_spam
    p_ham=prior_ham
    
    for word in email['words']:
        if word in flagged_words:
            try:
                #p_spam+=log(words[word]['p_spam']) #Log version - no longer used
                p_spam*=words[word]['p_spam']
            except ValueError:
                pass #This means that words that do not appear in a class will use the class prior
            try:
                #p_ham+=log(words[word]['p_ham']) #Log version - no longer used
                p_ham*=words[word]['p_ham']
            except ValueError:
                pass
    if p_spam>p_ham:
        y_pred=1
    else:
        y_pred=0
        
    y_true = email['y_true']
    if y_pred != y_true:
        miss+= 1.0
        
    sample_size += 1.0
    accuracy = ((sample_size-miss)/sample_size)*100
                
    print  '%30s' %email, '%10d' %y_true, '%10d' %y_pred, '%18.2f %%' % accuracy



In [ ]:
!chmod +x reducer.py

# Run Hadoop MapReduce Streaming

<h2> start up yarn and dfs </h2>

In [ ]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh       ### start up yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh        ### start up dfs

<h2> create folder </h2>

In [ ]:
!hdfs dfs -mkdir -p /user/venamax                       ### create hdfs folder

<h2> upload enronmail_1h.txt file </h2>

In [ ]:
!hdfs dfs -put enronemail_1h.txt /user/venamax                 #### save source data file to hdfs     

<h2> Hadoop streaming </h2>

In [ ]:
!hadoop jar hadoop-*streaming*.jar -mapper mapper.py -reducer reducer.py -input numcount.txt -output numcountOutput

<h2>HW2.2.1</h2>  Using Hadoop MapReduce and your wordcount job (from HW2.2) determine the top-10 occurring tokens (most frequent tokens)

<h2>HW2.3. Multinomial NAIVE BAYES with NO Smoothing</h2>
Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer job(s) that
   will both learn  Naive Bayes classifier and classify the Enron email messages using the learnt Naive Bayes classifier. Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). Note: for multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

   the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM 

   E.g.,   “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in all documents labeled as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. Note this is a multinomial estimation of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW. Multiplying lots of probabilities, which are between 0 and 1, can result in floating-point underflow. Since log(xy) = log(x) + log(y), it is better to perform all computations by summing logs of probabilities rather than multiplying probabilities. Please pay attention to probabilites that are zero! They will need special attention. Count up how many times you need to process a zero probabilty for each class and report. 

   Report the performance of your learnt classifier in terms of misclassifcation error rate of your multinomial Naive Bayes Classifier. Plot a histogram of the log posterior probabilities (i.e., Pr(Class|Doc))) for each class over the training set. Summarize what you see. 

   Error Rate = misclassification rate with respect to a provided set (say training set in this case). It is more formally defined here:

Let DF represent the evalution set in the following:
Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}|   / |DF|

Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by the Model “Model”

<h2>HW2.4 Repeat HW2.3 with the following modification: use Laplace plus-one smoothing. </h2>
Compare the misclassifcation error rates for 2.3 versus 2.4 and explain the differences.

For a quick reference on the construction of the Multinomial NAIVE BAYES classifier that you will code,
please consult the "Document Classification" section of the following wikipedia page:

https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Document_classification

OR the original paper by the curators of the Enron email data:

http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf

<h2>HW2.5. Repeat HW2.4. This time when modeling and classification ignore tokens with a frequency of less than three (3) in the training set. </h2>How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifier on the training dataset:

<h2>HW2.6 Benchmark your code with the Python SciKit-Learn implementation of the multinomial Naive Bayes algorithm</h2>

It always a good idea to benchmark your solutions against publicly available libraries such as SciKit-Learn, The Machine Learning toolkit available in Python. In this exercise, we benchmark ourselves against the SciKit-Learn implementation of multinomial Naive Bayes.  For more information on this implementation see: http://scikit-learn.org/stable/modules/naive_bayes.html more  

In this exercise, please complete the following:

— Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW2.5 and report the misclassification error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
- Prepare a table to present your results, where rows correspond to approach used (SkiKit-Learn versus your Hadoop implementation) and the column presents the training misclassification error
— Explain/justify any differences in terms of training error rates over the dataset in HW2.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn 



<h2>HHW 2.6.1 OPTIONAL (note this exercise is a stretch HW and optional)</h2>
—  Run the Bernoulli Naive Bayes algorithm from SciKit-Learn (using default settings) over the same training data used in HW2.6 and report the misclassification error 
-  Discuss the performance differences in terms of misclassification error rates over the dataset in HW2.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn. Why such big differences. Explain. 

Which approach to Naive Bayes would you recommend for SPAM detection? Justify your selection.

<h2>HW2.7 OPTIONAL (note this exercise is a stretch HW and optional)</h2>

The Enron SPAM data in the following folder enron1-Training-Data-RAW is in raw text form (with subfolders for SPAM and HAM that contain raw email messages in the following form:

--- Line 1 contains the subject
--- The remaining lines contain the body of the email message.

In Python write a script to produce a TSV file called train-Enron-1.txt that has a similar format as the enronemail_1h.txt that you have been using so far. Please pay attend to funky characters and tabs. Check your resulting formated email data in Excel and in Python (e.g., count up the number of fields in each row; the number of SPAM mails and the number of HAM emails). Does each row correspond to an email record with four values? Note: use "NA" to denote empty field values.

<h2>HW2.8 OPTIONAL</h2>
Using Hadoop Map-Reduce write job(s) to perform the following:
 -- Train a multinomial Naive Bayes Classifier with Laplace plus one smoothing using the data extracted in HW2.7 (i.e., train-Enron-1.txt). Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). Drop tokens with a frequency of less than three (3).
 -- Test the learnt classifier using enronemail_1h.txt and report the misclassification error rate. Remember to use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). How do we treat tokens in the test set that do not appear in the training set?

<h2>HW2.8.1 OPTIONAL</h2>
—  Run  both the Multinomial Naive Bayes and the Bernoulli Naive Bayes algorithms from SciKit-Learn (using default settings) over the same training data used in HW2.8 and report the misclassification error on both the training set and the testing set
- Prepare a table to present your results, where rows correspond to approach used (SciKit-Learn Multinomial NB; SciKit-Learn Bernouili NB; Your Hadoop implementation)  and the columns presents the training misclassification error, and the misclassification error on the test data set
-  Discuss the performance differences in terms of misclassification error rates over the test and training datasets by the different implementations. Which approch (Bernouili versus Multinomial) would you recommend for SPAM detection? Justify your selection.


<h2>=====================
END OF HOMEWORK</h2>